In [3]:
import pandas as pd

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

In [5]:
nndss_app_token = os.getenv('NNDSS_APP_TOKEN')

In [4]:
from datetime import datetime, timedelta

def get_year_week_from_date(date):
    """Returns the year and week number for a given date."""
    year, week, _ = date.isocalendar()
    return year, week

# Current date
current_date = datetime.now()

# First day of the current month
first_day_current_month = current_date.replace(day=1)

# Last day of the last month
last_day_last_month = first_day_current_month - timedelta(days=1)

# First day of the last month
first_day_last_month = last_day_last_month.replace(day=1)

# Get year and week number for the start and end of the last month
start_year, start_week = get_year_week_from_date(first_day_last_month)
end_year, end_week = get_year_week_from_date(last_day_last_month)

# Display the year and week range
print(f"Start of Last Month: Year = {start_year}, Week = {start_week}")
print(f"End of Last Month: Year = {end_year}, Week = {end_week}")


Start of Last Month: Year = 2024, Week = 5
End of Last Month: Year = 2024, Week = 9


In [75]:
# Calculate the previous month's MMWR week and year (This is a placeholder - you'll need to determine the actual logic based on MMWR calendar)
last_month = datetime.now().replace(day=1) - timedelta(days=1)
mmwr_week = last_month.strftime('%U')  # This is not the actual MMWR week, you'll need the MMWR calendar for this
mmwr_year = last_month.strftime('%Y')
# Construct the API URL with the MMWR week and year for the last month
# You'll need to replace 'MMWR_WEEK' and 'MMWR_YEAR' with actual fields from the dataset if they're different
url = f"https://data.cdc.gov/resource/x9gk-5huc.json?$$app_token={nndss_app_token}&$where=mmwr_week='{mmwr_week}' AND mmwr_year='{mmwr_year}'"

#print(url)

In [5]:
year = 2023
week = 2
state = 'CALIFORNIA'
label = 'Chlamydia trachomatis infection'

query_url = f"https://data.cdc.gov/resource/x9gk-5huc.json?$$app_token={nndss_app_token}&$where=year='{year}' AND states='{state}' AND label='{label}'"

#print(query_url)

In [7]:
import requests


In [8]:
# Make the API request
response = requests.get(query_url)
data = response.json()

In [9]:
data[0]

{'states': 'CALIFORNIA',
 'year': '2023',
 'week': '1',
 'label': 'Chlamydia trachomatis infection',
 'm1': '1765',
 'm1_flag': '-',
 'm2': '3939',
 'm2_flag': '-',
 'm3': '1765',
 'm3_flag': '-',
 'm4': '3368',
 'm4_flag': '-',
 'location1': 'CALIFORNIA',
 'sort_order': '20230101389',
 'geocode': {'type': 'Point', 'coordinates': [-121.49086, 38.57944]}}

In [10]:
df = pd.DataFrame(data)
df.head()

,states,year,week,label,m1,m1_flag,m2,m2_flag,m3,m3_flag,m4,m4_flag,location1,sort_order,geocode
0,CALIFORNIA,2023,1,Chlamydia trachomatis infection,1765,-,3939,-,1765,-,3368,-,CALIFORNIA,20230101389,"{'type': 'Point', 'coordinates': [-121.49086, ..."
1,CALIFORNIA,2023,2,Chlamydia trachomatis infection,2339,-,3948,-,5378,-,6727,-,CALIFORNIA,20230201389,"{'type': 'Point', 'coordinates': [-121.49086, ..."
2,CALIFORNIA,2023,3,Chlamydia trachomatis infection,2060,-,3950,-,9066,-,9885,-,CALIFORNIA,20230301389,"{'type': 'Point', 'coordinates': [-121.49086, ..."
3,CALIFORNIA,2023,4,Chlamydia trachomatis infection,2046,-,3950,-,11797,-,13510,-,CALIFORNIA,20230401389,"{'type': 'Point', 'coordinates': [-121.49086, ..."
4,CALIFORNIA,2023,5,Chlamydia trachomatis infection,2094,-,3950,-,14566,-,17308,-,CALIFORNIA,20230501389,"{'type': 'Point', 'coordinates': [-121.49086, ..."


In [1]:
# df 

In [13]:
base_url = "https://data.cdc.gov/resource/x9gk-5huc.json"

# Initialize parameters for pagination
limit = 50000  # Adjust based on your preference and the API's capabilities
offset = 0
data = []

while True:
    query_url = f"{base_url}?$$app_token={nndss_app_token}&$limit={limit}&$offset={offset}"
    response = requests.get(query_url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data: {response.status_code}")
        break
    
    batch = response.json()
    
    if not batch:
        # If the batch is empty, we've reached the end of the dataset
        break
    
    data.extend(batch)
    offset += limit
    print(f"Fetched {len(batch)} rows, total: {len(data)}")



Fetched 50000 rows, total: 50000
Fetched 50000 rows, total: 100000
Fetched 50000 rows, total: 150000
Fetched 50000 rows, total: 200000
Fetched 50000 rows, total: 250000
Fetched 50000 rows, total: 300000
Fetched 50000 rows, total: 350000
Fetched 50000 rows, total: 400000
Fetched 50000 rows, total: 450000
Fetched 50000 rows, total: 500000
Fetched 50000 rows, total: 550000
Fetched 50000 rows, total: 600000
Fetched 50000 rows, total: 650000
Fetched 50000 rows, total: 700000
Fetched 50000 rows, total: 750000
Fetched 50000 rows, total: 800000
Fetched 50000 rows, total: 850000
Fetched 21080 rows, total: 871080


In [14]:
# Convert to DataFrame
df = pd.DataFrame(data)
print(f"Total rows fetched: {len(df)}")

Total rows fetched: 871080


In [15]:
df.head()

,states,year,week,label,m1_flag,m2,m2_flag,m3_flag,m4_flag,location2,sort_order,location1,geocode,:@computed_region_hjsp_umg2,:@computed_region_skr5_azej,m4,m1,m3
0,US RESIDENTS,2022,1,Anthrax,-,0,-,-,-,US RESIDENTS,20220100001,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NEW ENGLAND,2022,1,Anthrax,-,0,-,-,-,NEW ENGLAND,20220100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CONNECTICUT,2022,1,Anthrax,-,0,-,-,-,NaN,20220100003,CONNECTICUT,"{'type': 'Point', 'coordinates': [-72.738288, ...",24,1043,NaN,NaN,NaN
3,MAINE,2022,1,Anthrax,-,0,-,-,-,NaN,20220100004,MAINE,"{'type': 'Point', 'coordinates': [-69.06137, 4...",49,1725,NaN,NaN,NaN
4,MASSACHUSETTS,2022,1,Anthrax,-,0,-,-,-,NaN,20220100005,MASSACHUSETTS,"{'type': 'Point', 'coordinates': [-71.481104, ...",25,1916,NaN,NaN,NaN


In [16]:
import requests
import json

base_url = "https://data.cdc.gov/resource/x9gk-5huc.json"


# Fetch the most recent records based on year and week, where location1 is not null
query_url = f"{base_url}?$$app_token={nndss_app_token}&$order=year DESC, week DESC&$where=location1 IS NOT NULL&$limit=1"

response = requests.get(query_url)

if response.status_code == 200:
    latest_record = response.json()
    if latest_record:
        # Extract the year and week from the most recent record
        latest_year = latest_record[0]['year']
        latest_week = latest_record[0]['week']
        
        # Now fetch all records for the most recent year and week, where location1 is not null
        week_data_query_url = f"{base_url}?$$app_token={nndss_app_token}&$where=year='{latest_year}' AND week='{latest_week}' AND location1 IS NOT NULL"
        week_data_response = requests.get(week_data_query_url)
        
        if week_data_response.status_code == 200:
            latest_week_data = week_data_response.json()
            print(f"Data for the most recent week of year {latest_year}, week {latest_week}, where location1 is not null: {json.dumps(latest_week_data[0], indent=2)}")
        else:
            print(f"Failed to fetch data for the latest week: {week_data_response.status_code}")
    else:
        print("No recent data found.")
else:
    print(f"Failed to fetch the latest data: {response.status_code}")


Data for the most recent week of year 2024, week 10, where location1 is not null: {
  "states": "CONNECTICUT",
  "year": "2024",
  "week": "10",
  "label": "Anthrax",
  "m1_flag": "-",
  "m2": "0",
  "m2_flag": "-",
  "m3_flag": "-",
  "m4_flag": "-",
  "location1": "CONNECTICUT",
  "sort_order": "20241000003",
  "geocode": {
    "type": "Point",
    "coordinates": [
      -72.738305908,
      41.57350273
    ]
  }
}


In [17]:
pd.DataFrame(latest_week_data)

,states,year,week,label,m1_flag,m2,m2_flag,m3_flag,m4_flag,location1,sort_order,geocode,m3,m4,m1
0,CONNECTICUT,2024,10,Anthrax,-,0,-,-,-,CONNECTICUT,20241000003,"{'type': 'Point', 'coordinates': [-72.73830590...",NaN,NaN,NaN
1,MAINE,2024,10,Anthrax,-,0,-,-,-,MAINE,20241000004,"{'type': 'Point', 'coordinates': [-69.20394706...",NaN,NaN,NaN
2,MASSACHUSETTS,2024,10,Anthrax,-,0,-,-,-,MASSACHUSETTS,20241000005,"{'type': 'Point', 'coordinates': [-71.718185, ...",NaN,NaN,NaN
3,NEW HAMPSHIRE,2024,10,Anthrax,-,0,-,-,-,NEW HAMPSHIRE,20241000006,"{'type': 'Point', 'coordinates': [-71.57759673...",NaN,NaN,NaN
4,RHODE ISLAND,2024,10,Anthrax,-,0,-,-,-,RHODE ISLAND,20241000007,"{'type': 'Point', 'coordinates': [-71.45431946...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,MARYLAND,2024,10,"Carbapenemase-Producing Organisms (CPO), Total",-,NaN,NC,-,-,MARYLAND,20241001223,"{'type': 'Point', 'coordinates': [-76.15873, 3...",NaN,NaN,NaN
996,NORTH CAROLINA,2024,10,"Carbapenemase-Producing Organisms (CPO), Total",-,NaN,NC,-,-,NORTH CAROLINA,20241001224,"{'type': 'Point', 'coordinates': [-79.18541791...",NaN,27,NaN
997,SOUTH CAROLINA,2024,10,"Carbapenemase-Producing Organisms (CPO), Total",-,NaN,NC,-,-,SOUTH CAROLINA,20241001225,"{'type': 'Point', 'coordinates': [-80.89375234...",NaN,NaN,NaN
998,VIRGINIA,2024,10,"Carbapenemase-Producing Organisms (CPO), Total",-,NaN,NC,-,-,VIRGINIA,20241001226,"{'type': 'Point', 'coordinates': [-78.69794871...",NaN,45,NaN


In [18]:
df.to_pickle("../data/raw/NNDSS.pkl")